<a href="https://colab.research.google.com/github/programminghumanity/iphs200fall2020/blob/main/twitter_step1_search_combine_20201128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gathering Tweets**

References:

* https://www.youtube.com/watch?v=_SqgSh3aR1g (20201127 10:17)
* https://null-byte.wonderhowto.com/how-to/mine-twitter-for-targeted-information-with-twint-0193853/ (20190502) 
* https://github.com/twintproject/twint/wiki (wiki)
* https://pielco11.ovh/posts/twint-osint/ (osint)

# **0. Setup Environment**

## You will need to give permission for this Colab to link to your gdrive in the code cell below

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# CUSTOMIZE: if you want your work and twitter datasets saved into a specific folder
#            beneath your gdrive root directory, define it below

dir_working = './MyDrive/courses/2020f_iphs200_programming_humanity/code/'

In [ ]:
# Move to working directory

%cd $dir_working

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code


In [ ]:
!git clone --depth=1 https://github.com/twintproject/twint.git


fatal: destination path 'twint' already exists and is not an empty directory.


In [ ]:
%cd twint


/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint


In [ ]:
# RESTART RUNTIME button at end of output (this takes a bit of time)

!pip3 install . -r requirements.txt

Processing /gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint
     |████████████████████████████████| 1.3MB 5.1MB/s 
     |████████████████████████████████| 266kB 33.4MB/s 
     |████████████████████████████████| 327kB 38.3MB/s 
     |████████████████████████████████| 296kB 45.7MB/s 
     |████████████████████████████████| 143kB 37.6MB/s 
     |████████████████████████████████| 235kB 43.4MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=6851ed29fa7c930810d1cdab79b909ece4f2699e23d874dea282042743516314
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for googletransx: filename=googletransx-2.4.2-cp36-none-any.whl size=15970 sha256=41ad2b346795e06897d348055c610348645805f14a4725140da26c098456da26
  Stored in directory: /root/.cache/pip/wheels/04/63/5f/75e7e94eb62517946116a783e4cd8970c478

## You should [RESTART RUNTIME] at this point before continuing

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
# CUSTOMIZE: if you want your work and twitter datasets saved into a specific folder
#            beneath your gdrive root directory, define it below

dir_working = './MyDrive/courses/2020f_iphs200_programming_humanity/code/'

In [ ]:
# Move to working directory

%cd $dir_working

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code


In [ ]:
# Move into the twint subdirectory created with the installtion of twint

%cd twint

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint


In [ ]:
!pwd

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
import glob

import time

In [ ]:
import twint

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# **1. DEFINE and RUN Searches**

In [ ]:
# LEAVE this blank, this is just defining a global
#   list variable that we'll populate below

search_terms_ls = []

## (a). Define your search_terms and/or user_name in the following code cell

In [ ]:
# STEP ONE: DEFINE search

# Define your search terms and/or user (you must set values for at least one of these, both ok too)

# CUSTOMIZE: If you want to limit your search to a particular user set 'username' to it
user_name = '' # uncomment this if you want to search by username
# user_name = '' # uncomment this line if you are not searching by username, only search terms

# CUSTOMIZE: If you want to search for key terms in the tweet, set 'search_terms'
search_terms = 'dallasmavs' # uncomment this if you want to search by key words, format 'term1 term2 term3'
                       #   all search terms should be separated by a single space within one string
# search_terms = '' # uncomment this line if you are not searching by any search terms, only username

In [ ]:
# STEP TWO: RUN search

# Create a filename of search results in a file with the name contained a unique timestamp
timestr = time.strftime("%Y%m%d-%H%M%S")
search_str = '_'.join(search_terms.split())
file_name = f"tweets_twint_{search_str}__{timestr}.csv"

# Save your user/search terms
search_terms_both = f"USERNAME:{user_name} and SEARCHTERMS:{search_terms}"
search_terms_ls.append(search_terms_both)

# Run your search
c = twint.Config()
if len(user_name) > 0:
  c.Username = user_name
if len(search_terms) > 0:
  c.Search = search_terms

if (len(user_name) != 0)  | (len(search_terms) != 0):
  c.Limit = 20000    # Limit of how many tweets to retrieve
  c.Store_object=True
  c.Store_csv = True
  c.Output = file_name # "tweets.csv"
  twint.run.Search(c)
else:
  print("You need to set user_name and/or search_terms to a non-null value")

Streaming output truncated to the last 5000 lines.
1329859629585666048 2020-11-20 18:50:17 +0000 <Johhnn__EE> @dallasmavs @luka7doncic we ready  https://t.co/hu5vRoIzmp
1329859473238847489 2020-11-20 18:49:40 +0000 <_whoisdez> @dallasmavs
1329859444214288389 2020-11-20 18:49:33 +0000 <zachattack1188> PLEASE @mcuban @dallasmavs
1329859359300530180 2020-11-20 18:49:13 +0000 <gerry_lemus> @dallasmavs @sixers @sdotcurry @jero114 nooooo no nos tocaba 😔👌
1329859182720405506 2020-11-20 18:48:31 +0000 <MmunhozdaR> @dallasmavs Announce Bogdan Bogdanovic
1329858730683469826 2020-11-20 18:46:43 +0000 <OmarOrozco93> @dallasmavs Make the Mavs regret trading you curry i still can't believe we gave you up but this is how business is in the nba
1329858695698780162 2020-11-20 18:46:35 +0000 <pete13213066> @dallasmavs @sixers @sdotcurry Wtf are you talking about! Glad to see that sometime shot maker but never when it counted! Gone dang stop acting like that was Steff curry !
1329858251765256192 2020-11-

## (b). Go back and run the last two code cells DEFINE SEARCH and RUN SEARCH until:

* You modify your searches to focus in on what you want to find out
* You have gathered enough tweets necessary for rigourous analysis

In [ ]:
!pwd

/gdrive/My Drive/courses/2020f_iphs200_programming_humanity/code/twint


In [ ]:
!ls -altr *.csv

-rw------- 1 root root   790248 Dec  4 19:29 tweets_twint_tesla__20201204-192926.csv
-rw------- 1 root root   782295 Dec  4 19:31 tweets_combined_20201204-193111.csv
-rw------- 1 root root   782295 Dec  4 19:31 tweets_combined_20201204-193123.csv
-rw------- 1 root root    40893 Dec  4 19:57 tweets_ner_sa__20201204-195723.csv
-rw------- 1 root root   790257 Dec  5 14:18 tweets_twint_tesla__20201205-141801.csv
-rw------- 1 root root  5665660 Dec  8 16:42 tweets_twint_stopthesteal__20201208-164240.csv
-rw------- 1 root root 28209455 Dec  8 16:46 tweets_twint_stopthesteal__20201208-164638.csv
-rw------- 1 root root    40822 Dec  9 00:24 tweets_ner_sa__20201209-002453.csv
-rw------- 1 root root   986297 Dec  9 19:36 cleaned_tweets_combined_20201204-193123.csv
-rw------- 1 root root  1372659 Dec 16 18:15 tweets_twint___20201216-181508.csv
-rw------- 1 root root 11658067 Dec 16 18:24 tweets_twint_dallasmavs__20201216-182040.csv


In [ ]:
!ls -al tweets_twint*

-rw------- 1 root root  1372659 Dec 16 18:15 tweets_twint___20201216-181508.csv
-rw------- 1 root root  5665660 Dec  8 16:42 tweets_twint_stopthesteal__20201208-164240.csv
-rw------- 1 root root 28209455 Dec  8 16:46 tweets_twint_stopthesteal__20201208-164638.csv
-rw------- 1 root root   790248 Dec  4 19:29 tweets_twint_tesla__20201204-192926.csv
-rw------- 1 root root   790257 Dec  5 14:18 tweets_twint_tesla__20201205-141801.csv


In [ ]:
# COPY AND PASTE the name of the datafile listed above into command below

tweets_df = pd.read_csv('tweets_twint_tesla__20201205-141801.csv')

In [ ]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1591 non-null   int64  
 1   conversation_id  1591 non-null   int64  
 2   created_at       1591 non-null   object 
 3   date             1591 non-null   object 
 4   time             1591 non-null   object 
 5   timezone         1591 non-null   int64  
 6   user_id          1591 non-null   int64  
 7   username         1591 non-null   object 
 8   name             1591 non-null   object 
 9   place            0 non-null      float64
 10  tweet            1591 non-null   object 
 11  language         1591 non-null   object 
 12  mentions         1591 non-null   object 
 13  urls             1591 non-null   object 
 14  photos           1591 non-null   object 
 15  replies_count    1591 non-null   int64  
 16  retweets_count   1591 non-null   int64  
 17  likes_count   

In [ ]:
# Try converting DataFrame.Series/columns to more specific types

tweets_df = tweets_df.convert_dtypes()
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1591 non-null   Int64  
 1   conversation_id  1591 non-null   Int64  
 2   created_at       1591 non-null   string 
 3   date             1591 non-null   string 
 4   time             1591 non-null   string 
 5   timezone         1591 non-null   Int64  
 6   user_id          1591 non-null   Int64  
 7   username         1591 non-null   string 
 8   name             1591 non-null   string 
 9   place            0 non-null      Int64  
 10  tweet            1591 non-null   string 
 11  language         1591 non-null   string 
 12  mentions         1591 non-null   string 
 13  urls             1591 non-null   string 
 14  photos           1591 non-null   string 
 15  replies_count    1591 non-null   Int64  
 16  retweets_count   1591 non-null   Int64  
 17  likes_count   

In [ ]:
tweets_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1334127507906314240,1333882505775050752,2020-12-02 13:29:19 UTC,2020-12-02,13:29:19,0,44196397,elonmusk,Elon Musk,<NA>,@Tesmanian_com Award accepted on behalf of the...,en,[],[],[],313,288,8495,[],[],https://twitter.com/elonmusk/status/1334127507...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'Tesmanian_com', 'name': 'Tes...",<NA>,<NA>,<NA>,<NA>
1,1331075177262661633,1330843965613027329,2020-11-24 03:20:27 UTC,2020-11-24,03:20:27,0,44196397,elonmusk,Elon Musk,<NA>,@PPathole @Teslarati @TeslaRoadTrip We’re stil...,en,[],[],[],135,97,2544,[],[],https://twitter.com/elonmusk/status/1331075177...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'PPathole', 'name': 'Pranay P...",<NA>,<NA>,<NA>,<NA>
2,1330982572038500355,1330980211509186560,2020-11-23 21:12:28 UTC,2020-11-23,21:12:28,0,44196397,elonmusk,Elon Musk,<NA>,@vincent13031925 @Tesla Wow,und,[],[],[],790,368,19407,[],[],https://twitter.com/elonmusk/status/1330982572...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'vincent13031925', 'name': 'V...",<NA>,<NA>,<NA>,<NA>
3,1330206139385044999,1330179916587884544,2020-11-21 17:47:12 UTC,2020-11-21,17:47:12,0,44196397,elonmusk,Elon Musk,<NA>,@heydave7 @philwhln Tesla is a vehicle for cre...,en,[],[],[],103,106,1941,[],[],https://twitter.com/elonmusk/status/1330206139...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'heydave7', 'name': 'Dave Lee...",<NA>,<NA>,<NA>,<NA>
4,1330187635483140099,1330179916587884544,2020-11-21 16:33:40 UTC,2020-11-21,16:33:40,0,44196397,elonmusk,Elon Musk,<NA>,@philwhln @heydave7 Because I am not an invest...,en,[],[],[],130,146,2497,[],[],https://twitter.com/elonmusk/status/1330187635...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'philwhln', 'name': 'Phil Whe...",<NA>,<NA>,<NA>,<NA>


In [ ]:
# how many tweets have a non-null URL field?

# len(tweets_df[tweets_df['quote_url'].str.len() > 0])

# **2. Aggregate your twitter datasets from all your searches and write to file (including a manifest file)**

In [ ]:
!ls tweets_twint*

tweets_twint_tesla__20201204-192926.csv
tweets_twint_tesla__20201205-141801.csv


In [ ]:
# CUSTOMIZE: Define a RegEx pattern that matches all the twitter dataset files 
#            you want to combine (e.g. tweets_twint_*.csv)

tweet_filenames = 'tweets_twint_*.csv'  # Define the RegEx that matches all your twitter dataset files

all_tweet_files = [i for i in glob.glob(tweet_filenames)] # You should use 'tweets_*.csv'
print(all_tweet_files)

['tweets_twint_tesla__20201204-192926.csv', 'tweets_twint_tesla__20201205-141801.csv']


In [ ]:
# Review user/search terms used to create these files

search_terms_ls

['USERNAME:elonmusk and SEARCHTERMS:tesla']

In [ ]:
# Combine all twitter dataset files into one pd.DataFrame
combined_df = pd.concat([pd.read_csv(f) for f in all_tweet_files ])

# Create a filename of search results in a file with the name contained a unique timestamp
timestr = time.strftime("%Y%m%d-%H%M%S")
file_name_combo = f"tweets_combined_{timestr}.csv"

# Write all tweets combined into one big file
combined_df.to_csv(file_name_combo, index=False, encoding='utf-8-sig')

# Write an accompnaying manifest file with all the search terms used to gather tweets in the combined file
file_name_combo_root = file_name_combo.split('.')[0]
file_name_combo_manifest = f"{file_name_combo_root}_manifest.txt"
with open(file_name_combo_manifest, 'w') as fp:
  fp.writelines(search_terms_ls)

In [ ]:
# CONFIRM: Both the 'tweets_combined_*.csv' and 'tweets_combined_*_manifest.txt' files exist
#          and that the size of the combined dataset file is approximately equal to all the 
#          individual dataset files that went into it

!ls -al tweets_*

-rw------- 1 root root 782295 Dec  4 19:31 tweets_combined_20201204-193111.csv
-rw------- 1 root root     39 Dec  4 19:31 tweets_combined_20201204-193111_manifest.txt
-rw------- 1 root root 782295 Dec  4 19:31 tweets_combined_20201204-193123.csv
-rw------- 1 root root     39 Dec  4 19:31 tweets_combined_20201204-193123_manifest.txt
-rw------- 1 root root  40893 Dec  4 19:57 tweets_ner_sa__20201204-195723.csv
-rw------- 1 root root 790248 Dec  4 19:29 tweets_twint_tesla__20201204-192926.csv
-rw------- 1 root root 790257 Dec  5 14:18 tweets_twint_tesla__20201205-141801.csv
-rw------- 1 root root   1273 Dec  4 19:58 tweets_wordcloud.png


In [ ]:
# CHECK: Make sure the manifest.txt file associated with our combined tweet dataset file saved all our
#        search terms

# type in the full filename of the most recent '*_manisfest.txt' file listed in the previous code cell above

!cat tweets_combined_20201201-012404_manifest.txt

cat: tweets_combined_20201201-012404_manifest.txt: No such file or directory


In [ ]:
combined_df = combined_df.convert_dtypes()
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1591 non-null   Int64  
 1   conversation_id  1591 non-null   Int64  
 2   created_at       1591 non-null   string 
 3   date             1591 non-null   string 
 4   time             1591 non-null   string 
 5   timezone         1591 non-null   Int64  
 6   user_id          1591 non-null   Int64  
 7   username         1591 non-null   string 
 8   name             1591 non-null   string 
 9   place            0 non-null      Int64  
 10  tweet            1591 non-null   string 
 11  language         1591 non-null   string 
 12  mentions         1591 non-null   string 
 13  urls             1591 non-null   string 
 14  photos           1591 non-null   string 
 15  replies_count    1591 non-null   Int64  
 16  retweets_count   1591 non-null   Int64  
 17  likes_count   

In [ ]:
!pwd

/content


In [ ]:
!ls

sample_data
tweets_combined_20201204-192048.csv
tweets_combined_20201204-192048_manifest.txt
tweets_twint_tesla__20201204-191635.csv
tweets_twint_tesla__20201204-191719.csv
tweets_twint_tesla__20201204-191954.csv


# **3. Clean tweets and separate out subcomponents (e.g. hashtags, emojis, etc)**

# **4. Exploratory Data Analysis (EDA) and Visualizations of Raw Data**